In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!git clone https://github.com/01pooja10/MobileGen

Cloning into 'MobileGen'...
remote: Enumerating objects: 291, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 291 (delta 57), reused 0 (delta 0), pack-reused 187
Receiving objects: 100% (291/291), 357.82 KiB | 21.05 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [3]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install python-multipart
!pip install wandb
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=ecf3975213b653

In [4]:
import sys
sys.path.append('/content/MobileGen/DDPM/scripts/')
sys.path.append('/content/drive/MyDrive/csc2231')

In [5]:
from fastapi import FastAPI, Response, Form
from fastapi.responses import StreamingResponse, FileResponse
from typing import Optional
import tempfile
import shutil
import os
from fastapi.middleware.cors import CORSMiddleware
from pathlib import Path
from PIL import Image
import PIL

from updated_inference import infer

In [10]:
app = FastAPI()

origins = [
    "*",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

def generate_image(txt: str, ver:int):

    classes = {'airplane':0, 'auto':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
    label = classes[txt]

    image = Image.fromarray(infer(label,ver)).resize((1024,1024))
    image_path = "/content/drive/MyDrive/csc2231/demo_outputs/gen_img.png"

    image.save(image_path)
    return image_path

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/generate_image")
async def get_image(text: str = Form(...), version: int = Form(...)):
    image_path = Path(generate_image(text, version))
    if not image_path.is_file():
        return {"error": "Image not found on the server"}
    return FileResponse(image_path)

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

auth_token = "2f6w5bNJItZ8FJ0rVNqeS23qGjj_6oaxarRcKRxN3vDBDYsaf"
ngrok.set_auth_token(auth_token)

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)